# Banana collector

---

In this notebook, we will test already trained DQN agent.

### 1. Start the Environment


In [ ]:
import sys
sys.path.append('../')

from unityagents import UnityEnvironment
from agents.dqn_agent import Agent

import config 
import torch

env = UnityEnvironment(file_name="Banana_Linux/Banana.x86_64")

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# Speed (False: Real time, True: Fast)
train_mode = False

# reset the environment
env_info = env.reset(train_mode=train_mode)[brain_name]

# Load parameters from file
params = config.HYPERPARAMS

# Create DQN agent instance
agent = Agent(params)

# Load learned model weight
agent.qnetwork_local.load_state_dict(torch.load('models/checkpoint.pth'))

### 2. Run DQN agent!

In [ ]:
# Number of episodes to run
n_episodes = 3

# Train loop
for i_episode in range(1, n_episodes+1):
    # Reset environment
    env_info = env.reset(train_mode=train_mode)[brain_name]

    # Observe current state
    state = env_info.vector_observations[0]

    # Reset score and done flag
    score = 0
    done = False

    # Episode loop
    while not done:

        # Select action with greedy policy
        action = agent.act(state)

        # Take action
        env_info = env.step(action)[brain_name]

        # Observe the next state
        next_state = env_info.vector_observations[0]

        # Get the reward
        reward = env_info.rewards[0]

        # Check if episode is finished
        done = env_info.local_done[0]

        # State transition
        state = next_state

        # Update total score
        score += reward

    # Print episode summary
    print('Episode %d Score:%d'%(i_episode, score))

# close the environment
env.close()